In [1]:
#!pip install syft

In [2]:
import syft as sy
from syft.core.node.new.util import autocache, PANDAS_DATA
sy.requires(">=0.8-beta")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


✅ The installed version of syft==0.8.0b6 matches the requirement >=0.8b0


In [3]:
!rm /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

rm: /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite: No such file or directory


In [4]:
node = sy.orchestra.launch(name="test-domain-1")

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.Data

In [5]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [6]:
# The usual preamble
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

plt.rcParams['figure.figsize'] = (15, 5)

# Load data

We're going to use a new dataset here, to demonstrate how to deal with larger datasets. This is a subset of the of 311 service requests from NYC Open Data.

In [7]:
# because of mixed types we specify dtype to prevent any errors
complaints = pd.read_csv(autocache(f"{PANDAS_DATA}/311-service-requests.csv"), dtype='unicode')

Depending on your pandas version, you might see an error like "DtypeWarning: Columns (8) have mixed types". This means that it's encountered a problem reading in our data. In this case it almost certainly means that it has columns where some of the entries are strings and some are integers.

For now we're going to ignore it and hope we don't run into a problem, but in the long run we'd need to investigate this warning.

In [8]:
complaints

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,26589651,10/31/2013 02:08:41 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,11432,90-03 169 STREET,169 STREET,90 AVENUE,91 AVENUE,NaN,NaN,ADDRESS,JAMAICA,NaN,Precinct,Assigned,10/31/2013 10:08:41 AM,10/31/2013 02:35:17 AM,12 QUEENS,QUEENS,1042027,197389,Unspecified,QUEENS,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.70827532593202,-73.79160395779721,"(40.70827532593202, -73.79160395779721)"
1,26593698,10/31/2013 02:01:04 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11378,58 AVENUE,58 AVENUE,58 PLACE,59 STREET,NaN,NaN,BLOCKFACE,MASPETH,NaN,Precinct,Open,10/31/2013 10:01:04 AM,NaN,05 QUEENS,QUEENS,1009349,201984,Unspecified,QUEENS,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.721040535628305,-73.90945306791765,"(40.721040535628305, -73.90945306791765)"
2,26594139,10/31/2013 02:00:24 AM,10/31/2013 02:40:32 AM,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,10032,4060 BROADWAY,BROADWAY,WEST 171 STREET,WEST 172 STREET,NaN,NaN,ADDRESS,NEW YORK,NaN,Precinct,Closed,10/31/2013 10:00:24 AM,10/31/2013 02:39:42 AM,12 MANHATTAN,MANHATTAN,1001088,246531,Unspecified,MANHATTAN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.84332975466513,-73.93914371913482,"(40.84332975466513, -73.93914371913482)"
3,26595721,10/31/2013 01:56:23 AM,10/31/2013 02:21:48 AM,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Horn,Street/Sidewalk,10023,WEST 72 STREET,WEST 72 STREET,COLUMBUS AVENUE,AMSTERDAM AVENUE,NaN,NaN,BLOCKFACE,NEW YORK,NaN,Precinct,Closed,10/31/2013 09:56:23 AM,10/31/2013 02:21:10 AM,07 MANHATTAN,MANHATTAN,989730,222727,Unspecified,MANHATTAN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.7780087446372,-73.98021349023975,"(40.7780087446372, -73.98021349023975)"
4,26590930,10/31/2013 01:53:44 AM,NaN,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,Vacant Lot,10027,WEST 124 STREET,WEST 124 STREET,LENOX AVENUE,ADAM CLAYTON POWELL JR BOULEVARD,NaN,NaN,BLOCKFACE,NEW YORK,NaN,NaN,Pending,11/30/2013 01:53:44 AM,10/31/2013 01:59:54 AM,10 MANHATTAN,MANHATTAN,998815,233545,Unspecified,MANHATTAN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.80769092704951,-73.94738703491433,"(40.80769092704951, -73.94738703491433)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111064,26426013,10/04/2013 12:01:13 AM,10/07/2013 04:07:16 PM,DPR,Department of Parks and Recreation,Maintenance or

# Create Mock data

In [9]:
from syft.core.node.new.mock import CachedFaker
from random import randint
import random

Let's create the mock data for the complaint dataset.

In [10]:
fake = CachedFaker()

In [11]:
fake_functions = {
    "Unique Key": lambda x: randint(1,1000000),
    "Location": lambda x: (fake.coordinate(), fake.coordinate()),
    "Agency": lambda x: random.choice(["NYPD", "DOHMH", "DPR"]),
    "X Coordinate (State Plane)": lambda x: randint(1,1000000),
    "Y Coordinate (State Plane)": lambda x: randint(1,1000000),
    "Complaint Type": lambda x: random.choice(["Illegal Parking", "Noise - Street/Sidewalk", "'Animal in a Park'"]),
    "Descriptor": lambda x: random.choice([ 'Branch or Limb Has Fallen Down','Branches Damaged','Broken Fence', 'Broken Glass']),
    "School Number": lambda x: random.choice([ 'B073', 'B077', 'B079', 'B080-01', 'B087', 'B099', 'B100', 'B102', 'B109', 'B111']),
    "Bridge Highway Segment": lambda x: random.choice([ 'Grand Central Pkwy (Exit 1 E-W)',
                                                       'Grand Central Pkwy (Exit 10) - 69th Rd-Jewel Ave (Exit 11)',
                                                       'GrandCentral Pkwy/VanWyck Expwy/College Point Blvd (Exit 22 A-E)',
                                                       'Hamilton Ave (Exit 2A) - Gowanus Expwy (I-278) (Exit 1)',
                                                       'Harding Ave (Exit 9) - Throgs Neck Br'])
}

In [12]:
fake_triggers = {
    "Street": lambda x : fake.street_name(),
    "Date": lambda x : fake.date_time(),
    "Long": lambda x : fake.coordinate(),
    "Lat": lambda x : fake.coordinate(),
    "Address": lambda x : fake.address(),
    "Name": lambda x : fake.name(),
    "City": lambda x : fake.city(),
    "Zip": lambda x : fake.zipcode(),
}

In [13]:
mock_data = dict()
for col in complaints.columns:
    col_vals = complaints[col]
    
    if col in fake_functions:
        mock_func = fake_functions[col]
    elif len(set(complaints[col])) < 100:
        values = list(set(complaints[col]))
        mock_func = lambda x: random.choice(values)
    else:
        for trigger in fake_triggers.keys():
            if trigger in col:
                mock_func = fake_triggers[trigger]
    mock_data[col] = [mock_func(None) for x in range(len(complaints))]

In [14]:
mock = pd.DataFrame(data=mock_data)

In [15]:
mock.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,504803,1997-12-21 06:54:53,1978-07-30 15:04:29,DPR,Sandy Keller,'Animal in a Park',Branch or Limb Has Fallen Down,School,25050,"49284 Darren Parks Apt. 211\nPort Devin, NM 97029",Anna Mullins,Matthew Crossing,Joshua Fort,Joseph Lane,Rebecca Ways,ADDRESS,South Erin,SI FERRY TERMINAL,DSNY Garage,Assigned,1987-03-22 01:30:13,2014-07-16 17:45:53,02 BROOKLYN,BROOKLYN,729625,488107,William Davis,Unspecified,Maurice Hall,B080-01,Region 8,28Q160,2124080250,USCGC Ayers\nFPO AP 14340,HOWARD BEACH,NY,56222,NaN,NaN,NaN,BRONX,NaN,NaN,Eastbound,Ramp,Harding Ave (Exit 9) - Throgs Neck Br,Far Rockaway #2 Field,NaN,JEWISH ASSOCIATION FOR SERVICES FOR THE AGED (...,-70.225696,-175.691661,"(108.354661, 5.244617)"
1,580809,2012-04-24 22:10:10,2008-03-03 23:13:05,DPR,Cassandra Sanders,Noise - Street/Sidewalk,Broken Fence,Other (explain in Complaint Details),93141,"8761 Mayo Path Suite 226\nLake Jacqueline, LA ...",Maurice Hall,Jennifer View,Ruiz Points,Kimberly Prairie,Smith Rue,LATLONG,New Emilyberg,PORT AUTH 42 STREET,Precinct,Assigned,2004-01-14 10:33:07,2012-04-24 22:10:10,55 BROOKLYN,STATEN ISLAND,652224,684447,Edward Vang DDS,STATEN ISLAND,Jessica Edwards,B087,Region 8,10X660,7189658973,"3454 Jessica Shoals Suite 165\nNorth Gregory, ...",HOWARD BEACH,NY,43927,Y,NaN,Car Service,BRONX,Port Authority Bus Terminal,FDR Dr,West/Toward Triborough Br,Ramp,Harding Ave (Exit 9) - Throgs Neck Br,Far Rockaway #2 Field,Staten Island Bound,IRA WEINSTEIN,-116.358749,-63.095821,"(178.776026, 55.882875)"
2,753216,1985-07-30 18:17:26,1993-11-24 02:17:04,DOHMH,David Barnett,'Animal in a Park',Broken Fence,Store/Commercial,65716,"8804 Scott Forks Apt. 511\nBooneburgh, IN 17496",Stephen Edwards,Davenport Ranch,Myers Lock,Brown Crest,Hunt Plains,NaN,Rileyview,SI FERRY TERMINAL,School,Started,1981-07-30 15:23:27,1972-01-24 10:02:30,64 MANHATTAN,Unspecified,689194,162147,Jesse Franco,QUEENS,Lindsey Murray,B111,NaN,22K139,7189658900,"6678 Brown Station\nPort Rebecca, IN 02392",FOREST HILLS,Unspecified,45693,NaN,NaN,Commuter Van,BROOKLYN,Intersection,Prospect Expwy,North/Westchester County Bound,Roadway,Grand Central Pkwy (Exit 1 E-W),White Plains Road Field,NaN,Molinari,118.941423,-152.828137,"(-129.871898, -2.675889)"
3,827745,1982-07-29 21:28:48,2014-03-09 20:35:31,DOHMH,Lisa Allison,'Animal in a Park',Broken Glass,NaN,55577,"512 Vasquez Turnpike\nNicoleside, HI 64372",Sandy Keller,Howard Estate,Maria Spur,Christina Way,Taylor Summit,INTERSECTION,Lake Matthew,JFK,Precinct,Unassigned,2011-09-16 07:08:26,2004-01-14 10:33:07,12 BRONX,BROOKLYN,336388,585355,Tanya Watson,QUEENS,Holly Leon,B109,Unspecified,21K226,7183537853,"22691 Mary Mills Apt. 092\nScottstad, FM 35863",ASTORIA,NY,89291,N,NaN,Commuter Van,NaN,La Guardia Airport,Washington Br/181 Street Br.,Brooklyn Bound,NaN,Grand Central Pkwy (Exit 1 E-W),Broome and Ludlow Field,Staten Island Bound,HOWARD BEACH SENIOR CENTER,54.995201,-150.423963,"(-9.467789, -78.315292)"
4,530222,2011-11-14 14:10:30,2008-07-27 17:51:38,DOHMH,Michael Juarez,Illegal Parking,Broken Fence,Catering Hall,54361,"178 Rivera Ports\nTracytown, GU 59651",Dylan Bryant,Matthew Mountains,Delgado Tunnel,Patrick Fields,Taylor Drive,LATLONG,West Sabrina,MADISON SQUARE G

In [ ]:
dataset = sy.Dataset(name="test",
                     asset_list=[sy.Asset(name="complaints", data=complaints, mock=mock, mock_is_real=False)])
domain_client.upload_dataset(dataset)


  0%|                                                                                                                                                                | 0/1 [00:00<?, ?it/s]

Uploading: complaints


# Create data scientist

In [ ]:
user = domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

# todo: give user data scientist role

guest_domain_client = node.client

guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")